In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
test = "mito_mem (3)"
print(test.split("(")[0].split(" ")[0])
for row in mi.rows:
    print(row.organelle_info)

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
from annotator_metrics.src.preprocessing import copy_data
mi = MaskInformation()
print(mi.rows)
copy_data(group = "group1", output_base_path="/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/")

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
print(mi.rows[0].organelle_info)


In [ ]:
import numpy as np
a=np.random.random((4,4,4))

In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
a=MaskInformation()
print(a.rows[1].organelle_info)

In [1]:
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
a,b = calculate_all_to_all("group1","/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/", "Mean False Distance (Clipped)",num_workers = 10)

In [8]:
from annotator_metrics.src.metrics_evaluations import compare_two_images
f1_score = compare_two_images(
    gt_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop171/jrc_mus-liver_crop171_ariadne_manual.tif',
    test_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop171/jrc_mus-liver_crop171_ariadne_refined.tif',
    organelle_label = 2,
    resolution = 8,
    metrics_to_calculate = "F1 Score")
print(f1_score)

f1_score = compare_two_images(
    gt_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop172/jrc_mus-liver_crop172_final_manual.tif',
    test_path = '/groups/cosem/cosem/annotations/amira/jrc_mus-liver/crop172/jrc_mus-liver_crop172_final_ariadne.tif',
    organelle_label = 2,
    resolution = 8,
    metrics_to_calculate = "F1 Score")
print(f1_score)


[[<EvaluationMetrics.dice: 'dice'>, 0.6412835166041323], [<EvaluationMetrics.f1_score: 'f1_score'>, 0.6412835166041322]]


In [1]:
from annotator_metrics.util.image_io import create_variance_images

create_variance_images("/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp","group1", "/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/n5s")

distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-4iqgagyz', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-i5e4visw', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-n78syz82', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-k_zuv4q0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-t8isiwu4', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/

http://10.150.100.248:8787/status


/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/zarr/n5.py:711: RuntimeWarning: Not all N5 implementations support blosc compression (yet). You might not be able to open the dataset with another N5 library.
  RuntimeWarning
/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/zarr/n5.py:711: RuntimeWarning: Not all N5 implementations support blosc compression (yet). You might not be able to open the dataset with another N5 library.
  RuntimeWarning
/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/zarr/n5.py:711: RuntimeWarning: Not all N5 implementations support blosc compression (yet). You might not be able to open the dataset with another N5 library.
  RuntimeWarning
/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/zarr/n5.py:711: RuntimeWarning: Not all N5 implementations support blosc compression

In [4]:
import h5py
with h5py.File("/groups/cosem/cosem/annotations/amira//jrc_mus-liver/crop137/jrc_mus_liver_crop137_mito003.h5") as f:
    print(f["volumes"]["raw"].attrs["resolution"])
    print(f["volumes"]["labels"]["gt"].attrs["resolution"])



[4. 4. 4.]
[4. 4. 4.]


In [17]:
1023/2

511.5